In [2]:
from google.cloud import storage

storage_client = storage.Client()

In [3]:
bucket = storage_client.get_bucket("audl-heroku-data")
blob = bucket.blob("test/README.md")
blob.upload_from_filename("README.md")

In [4]:
bucket = storage_client.get_bucket("audl-heroku-data")
blob = bucket.blob("test/README.md")
blob.download_to_filename("README2.md")

In [1]:
%load_ext autoreload

In [66]:
%autoreload
import audl_advanced_stats as audl

# Examples

This notebook gives a brief explanation and examples of how to use the `audl_advanced_stats` package.

The first time you run these functions, the data will be downloaded and stored locally, which will take a while. When you run the functions after that, the locally stored versions will be used, which is much quicker. The times in this notebook reflect the first run-through.

In [18]:
import audl_advanced_stats as audl

## Season

This section will go through the `Season` class.

In [10]:
%%time
# Initialize season object
s = audl.Season(
    year=2023,
    data_path="data",
    upload=False,
    download=False
)

# Get all players
s.get_players().query("player_name=='Laviolette, Allan'").head()

CPU times: user 44.5 s, sys: 43.7 s, total: 1min 28s
Wall time: 1min 20s


,year,player_id,player_season_id,player_name,team_id
1142,2021,77,8815,"Laviolette, Allan",18


In [11]:
%%time
# Get all teams
s.get_teams(keep_all_years=True)

CPU times: user 515 ms, sys: 14.1 ms, total: 529 ms
Wall time: 529 ms


,year,team_id,team_abbrev,team_name
0,2023,1,ATL,Atlanta Hustle
1,2023,2,ATX,Austin Sol
2,2023,37,BOS,Boston Glory
3,2023,18,CAR,Carolina Flyers
4,2023,3,CHI,Chicago Union
5,2023,39,COL,Colorado Summit
6,2023,5,DC,DC Breeze
7,2023,4,DAL,Dallas Legion
8,2023,6,DET,Detroit Mechanix
9,2023,41,HTX,Houston Havoc


In [12]:
%%time
# Get all scheduled games and whether or not they've been played yet
s.get_game_info(keep_all_years=True).head()

CPU times: user 74 µs, sys: 0 ns, total: 74 µs
Wall time: 77.2 µs


,year,game_date,away_team,home_team,url,events_exist,playoffs
0,2023,2023-04-28,SEA,SLC,https://www.backend.audlstats.com/stats-pages/...,True,False
1,2023,2023-04-29,ATX,HTX,https://www.backend.audlstats.com/stats-pages/...,True,False
2,2023,2023-04-29,DC,CAR,https://www.backend.audlstats.com/stats-pages/...,True,False
3,2023,2023-04-29,IND,ATL,https://www.backend.audlstats.com/stats-pages/...,True,False
4,2023,2023-04-29,MAD,PIT,https://www.backend.audlstats.com/stats-pages/...,True,False


In [13]:
s.get_games().query("t==22").groupby(["game_id", "team_id"]).size().sort_values()

game_id  team_id
2976     37          5
2965     16          8
2976     14          8
2845     17          8
2842     23          8
                    ..
2850     3          33
2855     12         33
2916     39         34
2674     11         34
2932     21         36
Length: 764, dtype: int64

In [56]:
(
    s.get_games()
    .query("event_name==['In-bounds Pull', 'Out-of-bounds Pull']")
    ["t_after"].value_counts()
)

Series([], Name: t_after, dtype: int64)

In [34]:
import numpy as np
(
    s.get_games()
    .query("event_name==['In-bounds Pull', 'Out-of-bounds Pull']")
    .query("possession_outcome_general==['Score', 'Turnover']")
    .assign(event_name=lambda x: np.where(
        (x["y_after"]>100) & (x["event_name"]=="In-bounds Pull"),
        "In-bounds Pull: In End Zone",
        x["event_name"]
    ))
    .assign(event_name=lambda x: np.where(
        (x["y_after"]>80) & (x["y_after"]<=100) & (x["event_name"]=="In-bounds Pull"),
        "In-bounds Pull: Brick to End Zone",
        x["event_name"]
    ))
    .assign(event_name=lambda x: np.where(
        (x["y_after"]<=80) & (x["event_name"]=="In-bounds Pull"),
        "In-bounds Pull Before Brick",
        x["event_name"]
    ))
    .groupby(["year", "event_name", "possession_outcome_general"])
    .size()
    .groupby(level=[0,1])
    .apply(lambda x: x/x.sum())
    .rename("first_possession_score_pct")
    .reset_index()
    .assign(year=lambda x: x["year"].astype(int))
    .query("possession_outcome_general=='Score'")
    .drop(columns=["possession_outcome_general"])
    .set_index(["year", "event_name"])
    .unstack(level=["event_name"])
    .reset_index()
)

year  first_possession_score_pct  \
event_name       In-bounds Pull Before Brick   
0           2021                    0.518201   
1           2022                    0.559854   
2           2023                    0.491713   

                                                                          \
event_name In-bounds Pull: Brick to End Zone In-bounds Pull: In End Zone   
0                                   0.534173                    0.530885   
1                                   0.543275                    0.518610   
2                                   0.514706                    0.477499   

                               
event_name Out-of-bounds Pull  
0                    0.554688  
1                    0.539130  
2                    0.542435

In [40]:
import numpy as np
pull_pct = (
    s.get_games()
    .query("event_name==['In-bounds Pull', 'Out-of-bounds Pull']")
    .query("possession_outcome_general==['Score', 'Turnover']")
    .assign(event_name=lambda x: np.where(
        (x["y_after"]>100) & (x["event_name"]=="In-bounds Pull"),
        "In-bounds Pull: In End Zone",
        x["event_name"]
    ))
    .assign(event_name=lambda x: np.where(
        (x["y_after"]>80) & (x["y_after"]<=100) & (x["event_name"]=="In-bounds Pull"),
        "In-bounds Pull: Brick to End Zone",
        x["event_name"]
    ))
    .assign(event_name=lambda x: np.where(
        (x["y_after"]<=80) & (x["event_name"]=="In-bounds Pull"),
        "In-bounds Pull Before Brick",
        x["event_name"]
    ))
    .groupby(["year", "event_name"])
    .size()
    .groupby(level=[0])
    .apply(lambda x: x/x.sum())
    .rename("pct_of_pulls")
    .reset_index()
    .assign(year=lambda x: x["year"].astype(int))
    # .query("possession_outcome_general=='Score'")
    # .drop(columns=["possession_outcome_general"])
    .set_index(["year", "event_name"])
    .unstack(level=["event_name"])
    .reset_index()
    # .transform(lambda x: '{:,.2%}'.format(x))
)
pull_pct

year                pct_of_pulls  \
event_name       In-bounds Pull Before Brick   
0           2021                    0.243103   
1           2022                    0.194133   
2           2023                    0.150624   

                                                                          \
event_name In-bounds Pull: Brick to End Zone In-bounds Pull: In End Zone   
0                                   0.482388                    0.207878   
1                                   0.455151                    0.285532   
2                                   0.188627                    0.585576   

                               
event_name Out-of-bounds Pull  
0                    0.066632  
1                    0.065184  
2                    0.075173

In [14]:
%%time
# Compile all processed game data into a single dataframe
s.get_games().head()

CPU times: user 107 µs, sys: 91 µs, total: 198 µs
Wall time: 209 µs


,t,r,x,y,s,q,year,game_id,team_id,opponent_team_id,...,9607,8851,9307,8849,9202,9724,9725,9726,9727,9340
0,50,NaN,NaN,NaN,720.0,NaN,2023.0,3097,40,22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,NaN,NaN,NaN,NaN,2023.0,3097,40,22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20,11413.0,0.00,40.00,NaN,NaN,2023.0,3097,40,22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20,11421.0,-0.21,50.99,NaN,NaN,2023.0,3097,40,22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20,11413.0,-9.14,48.74,NaN,NaN,2023.0,3097,40,22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
%%time
# Get some basic QC info to determine where the raw data might be messed up
# This will typically take at least ~2-3 minutes
s.get_game_qc()

CPU times: user 1min 45s, sys: 2.58 s, total: 1min 48s
Wall time: 1min 46s


,year,game_date,away_team,home_team,url,events_exist,playoffs,not_seven_players_home,not_seven_players_away,not_seven_players_home_raw,not_seven_players_away_raw,negative_elapsed_home,negative_elapsed_away,negative_time_home_raw,negative_time_away_raw,no_event_label_home,no_event_label_away,data_labels_home_raw,data_labels_away_raw
0,2023,2023-04-28,SEA,SLC,https://www.backend.audlstats.com/stats-pages/...,True,False,0,0,0,0,3,0,1,0,[],[],"[l, ms, q, r, s, t, x, y]","[h, l, ms, q, r, s, t, x, y]"
1,2023,2023-04-29,ATX,HTX,https://www.backend.audlstats.com/stats-pages/...,True,False,1,0,0,0,0,0,0,0,[],[],"[l, ms, q, r, s, t, x, y]","[c, l, ms, r, s, t, x, y]"
2,2023,2023-04-29,DC,CAR,https://www.backend.audlstats.com/stats-pages/...,True,False,0,0,0,0,1,0,0,0,[],[],"[l, ms, q, r, s, t, x, y]","[h, l, ms, q, r, s, t, x, y]"
3,2023,2023-04-29,IND,ATL,https://www.backend.audlstats.com/stats-pages/...,True,False,0,0,0,0,0,1,0,0,[],[],"[c, l, ms, r, s, t, x, y]","[l, ms, r, s, t, x, y]"
4,2023,2023-04-29,MAD,PIT,https://www.backend.audlstats.com/stats-pages/...,True,False,0,0,0,0,0,0,0,0,[],[],"[c, l, ms, r, s, t, x, y]","[c, h, l, ms, r, s, t, x, y]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377,2021,2021-09-04,TOR,OTT,https://www.backend.audlstats.com/stats-pages/...,True,False,1,0,0,0,0,0,0,0,[],[],"[c, h, l, ms, r, s, t, x, y]","[c, l, ms, r, s, t, x, y]"
378,2021,2021-09-09,TOR,MTL,https://www.backend.audlstats.com/stats-pages/...,True,True,0,0,0,0,0,0,0,0,[],[],"[l, ms, r, s, t, x, y]","[l, ms, r, s, t, x, y]"
379,2021,2021-09-10,RAL,CHI,https://www.backend.audlstats.com/stats-pages/...,True,True,0,0,0,0,0,0,0,0,[],[],"[c, l, lr, ms, o, r, s, t, x, y]","[c, h, l, lr, ms, o, q, r, s, t, x, y]"
380,2021,2021-09-10,SD,NY,https://www.backend.audlstats.com/stats-pages/...,True,True,0,0,0,0,0,0,0,0,[],[],"[l, ms, r, s, t, x, y]","[c, l, ms, r, s, t, x, y]"


In [16]:
%%time
# Player stats for each separate game
s.get_player_stats_by_game().head()

/Users/johnlithio/Documents/AUDL-Advanced-Stats/audl_advanced_stats/game.py:153: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  dfout[f"receptions_{throw_type}_pct"] = dfout[


CPU times: user 13min 28s, sys: 7.67 s, total: 13min 35s
Wall time: 13min 37s


,player_season_id,playerid,name,team,opponent,game_date,year,playoffs,games,total_points,...,receptions_dump_pct,completion_huck_pct,attempts_huck_pct,receptions_huck_pct,completion_swing_pct,attempts_swing_pct,receptions_swing_pct,completion_throw_pct,attempts_throw_pct,receptions_throw_pct
0,11406,2657,Jacob Miller,SLC,SEA,2023-04-28,2023.0,False,1.0,23.0,...,0.000000,NaN,0.000000,0.0,1.0,0.166667,0.210526,1.000000,0.566667,0.473684
1,11407,2985,Nathan Huff,SLC,SEA,2023-04-28,2023.0,False,1.0,19.0,...,0.125000,NaN,0.000000,0.0,1.0,0.125000,0.375000,1.000000,0.625000,0.250000
2,11408,2986,Luke Yorgason,SLC,SEA,2023-04-28,2023.0,False,1.0,20.0,...,0.166667,1.0,0.047619,0.0,1.0,0.095238,0.305556,0.965517,0.690476,0.305556
3,11410,2987,Devon Terry,SLC,SEA,2023-04-28,2023.0,False,1.0,9.0,...,0.000000,NaN,0.000000,0.0,NaN,0.000000,0.000000,NaN,0.000000,1.000000
4,11411,2988,Jensen Wells,SLC,SEA,2023-04-28,2023.0,False,1.0,14.0,...,0.000000,NaN,0.000000,0.0,1.0,0.125000,0.333333,1.000000,0.625000,0.166667


In [17]:
%%time
# Aggregated player stats for the whole season
s.get_player_stats_by_season().query("name=='John Lithio'").head()

CPU times: user 83.8 ms, sys: 25.1 ms, total: 109 ms
Wall time: 143 ms


,playerid,player_season_id,name,team,year,games,total_points,o_points,o_point_scores,o_point_noturns,...,receptions_dump_pct,completion_huck_pct,attempts_huck_pct,receptions_huck_pct,completion_swing_pct,attempts_swing_pct,receptions_swing_pct,completion_throw_pct,attempts_throw_pct,receptions_throw_pct
1210,2639,10882,John Lithio,NY,2023.0,8.0,121.0,116.0,87.0,76.0,...,0.048611,0.666667,0.021127,0.027778,1.0,0.204225,0.138889,0.888889,0.380282,0.562500
1211,2639,9466,John Lithio,NY,2021.0,14.0,283.0,281.0,210.0,185.0,...,0.004202,0.500000,0.030612,0.016807,1.0,0.209184,0.025210,0.971429,0.357143,0.869748
1212,2639,9817,John Lithio,NY,2022.0,13.0,251.0,234.0,191.0,174.0,...,0.011538,0.666667,0.027778,0.026923,1.0,0.194444,0.084615,0.923077,0.421296,0.750000


In [18]:
s.get_player_stats_by_season(playoffs=True).head()
s.get_player_stats_by_season(playoffs=False).head()
s.get_start_of_opoints().head()

,year,game_id,team_id,opponent_team_id,period,t,t_after,r,r_after,x,...,s_before,o_point,possession_outcome_general,throw_outcome,yyards,yyards_raw,xyards,xyards_raw,yards,yards_raw
1,2023.0,3097,40,22,1,1,NaN,NaN,NaN,NaN,...,720.0,True,Score,None,NaN,NaN,NaN,NaN,NaN,NaN
38,2023.0,3097,40,22,1,1,NaN,NaN,NaN,NaN,...,0.0,True,Turnover,None,NaN,NaN,NaN,NaN,NaN,NaN
44,2023.0,3097,40,22,1,1,NaN,NaN,NaN,NaN,...,472.0,True,Score,None,NaN,NaN,NaN,NaN,NaN,NaN
65,2023.0,3097,40,22,1,1,NaN,NaN,NaN,NaN,...,375.0,True,Score,None,NaN,NaN,NaN,NaN,NaN,NaN
79,2023.0,3097,40,22,1,1,NaN,NaN,NaN,NaN,...,279.0,True,Score,None,NaN,NaN,NaN,NaN,NaN,NaN


In [67]:
%%time
# Heatmap and histograms
fighm, fighy, fighx = s.visual_field_heatmap_horizontal(
    outcome_measure="possession_outcome_general",
    outcome="Score",
    metric="pct",
    pull_only=True,
    remove_ob_pull=True,
    years=[2023]
)

# Combine heatmap and histograms into single plot
# The aspect ratio is weird here because it's handled in the app using html/css
s.visual_field_heatmap_subplots_horizontal(fighm, fighy, fighx)

CPU times: user 317 ms, sys: 146 ms, total: 463 ms
Wall time: 520 ms


## Game

This section will go through the `Game` class using the first game of the season (DC at NY) as an example.

In [28]:
g = audl.Game("https://audl-stat-server.herokuapp.com/stats-pages/game/2022-06-03-CAR-DAL")
g.get_player_stats_by_game(home=True).head()

,player_season_id,playerid,name,team,opponent,game_date,year,playoffs,games,total_points,...,receptions_dump_pct,completion_huck_pct,attempts_huck_pct,receptions_huck_pct,completion_swing_pct,attempts_swing_pct,receptions_swing_pct,completion_throw_pct,attempts_throw_pct,receptions_throw_pct
0,10177,220,Brandon Malecek,DAL,CAR,2022-06-03,2022.0,False,1.0,21.0,...,0.222222,1.000000,0.043478,0.000000,1.0,0.021739,0.194444,0.964286,0.608696,0.305556
1,10178,1966,Griffin Miller,DAL,CAR,2022-06-03,2022.0,False,1.0,17.0,...,0.000000,NaN,0.000000,0.000000,1.0,0.200000,0.000000,1.000000,0.400000,0.800000
2,10182,2186,Jimmy Zuraw,DAL,CAR,2022-06-03,2022.0,False,1.0,22.0,...,0.289474,0.666667,0.062500,0.000000,1.0,0.145833,0.105263,1.000000,0.541667,0.236842
3,10183,1287,Michael Matthis,DAL,CAR,2022-06-03,2022.0,False,1.0,22.0,...,0.000000,NaN,0.000000,0.136364,1.0,0.050000,0.090909,1.000000,0.250000,0.545455
4,10184,179,Ben Lewis,DAL,CAR,2022-06-03,2022.0,False,1.0,21.0,...,0.000000,NaN,0.000000,0.100000,1.0,0.307692,0.050000,1.000000,0.153846,0.750000


In [12]:
%%time
# Initialize game object using URL from season object
g = audl.Game(s.get_game_info()["url"].iloc[0])

CPU times: user 238 µs, sys: 40 µs, total: 278 µs
Wall time: 262 µs


In [13]:
%%time
# Here's what the raw events look like
# Each dict in the list represents a different event. For example, 20 is a completion, 21 is a score, etc.
# See constants.py for a list of the conversions.
g.get_home_events_raw()

CPU times: user 83.1 ms, sys: 1.87 ms, total: 85 ms
Wall time: 84.5 ms


[{'t': 50},
 {'t': 1, 'l': [10141, 10147, 10149, 10154, 10158, 10159, 10174]},
 {'t': 20, 'r': 10159, 'x': 13.64, 'y': 19.99},
 {'t': 20, 'r': 10141, 'x': -2.26, 'y': 41.59},
 {'t': 20, 'r': 10149, 'x': 5.93, 'y': 48.81},
 {'t': 20, 'r': 10141, 'x': 8.52, 'y': 38.52},
 {'t': 20, 'r': 10154, 'x': 23.25, 'y': 44.11},
 {'t': 20, 'r': 10141, 'x': 0.13, 'y': 39.61},
 {'t': 20, 'r': 10174, 'x': -16.03, 'y': 45.88},
 {'t': 20, 'r': 10149, 'x': -19.92, 'y': 61.15},
 {'t': 20, 'r': 10141, 'x': -2.05, 'y': 51.2},
 {'t': 22, 'r': 10158, 'x': 24.75, 'y': 108.61, 's': 678},
 {'t': 2, 'l': [10153, 10160, 10162, 10171, 10167, 10176, 10157]},
 {'t': 3, 'x': 0, 'y': 80, 'ms': 0},
 {'t': 5, 'r': 10160, 'q': 1},
 {'t': 20, 'r': 10160, 'x': -9.96, 'y': 65.52},
 {'t': 20, 'r': 10162, 'x': -7.37, 'y': 76.09},
 {'t': 20, 'r': 10171, 'x': 21.68, 'y': 82.56},
 {'t': 20, 'r': 10162, 'x': 22.64, 'y': 87.06},
 {'t': 20, 'r': 10171, 'x': 6.61, 'y': 90},
 {'t': 8, 'x': -23.81, 'y': 98.31},
 {'t': 11},
 {'t': 21, 's

In [14]:
%%time
# Here's what the processed events look like
g.get_home_events().head()

CPU times: user 5.65 ms, sys: 2.65 ms, total: 8.3 ms
Wall time: 5.28 ms


,t,r,x,y,s,q,h,game_id,team_id,opponent_team_id,...,10165,10166,10167,10171,10173,10174,10176,centering_pass,throw_type,play_description
0,50,NaN,NaN,NaN,720.0,NaN,NaN,2893,18,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,None,
1,1,NaN,NaN,NaN,NaN,NaN,NaN,2893,18,1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,False,None,
2,20,10159.0,13.64,19.99,NaN,NaN,NaN,2893,18,1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,True,Throw,Completion: Matt Gouchoe-Hanas Throw to<br>Sol...
3,20,10141.0,-2.26,41.59,NaN,NaN,NaN,2893,18,1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,False,Dish,Completion: Sol Yanuck Dish to<br>Henry Fisher...
4,20,10149.0,5.93,48.81,NaN,NaN,NaN,2893,18,1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,False,Dump,Completion: Henry Fisher Dump to<br>Sol Yanuck...


In [15]:
%%time
# We can also get some basic info about the game
g.get_game_info()

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 8.11 µs


,id,team_season_id_home,team_season_id_away,status_id,score_home,score_away,live,reg_season,ignore_game,lock,...,aw_section,ext_game_id,update_timestamp,location_id,ls_game_id,ticket_url,streaming_url,roster_set,roster_set_1,roster_set_2
0,2893,242,241,100,16,14,None,True,False,True,...,week-1,2022-04-29-ATL-CAR,2022-04-30T00:55:50.838Z,53,55790353-0270-4b7c-b916-948dd69b9abb,https://theaudl.com/flyers/tickets,https://audltv.vhx.tv/videos/atlanta-at-caroli...,True,True,True


In [16]:
%%time
# We can also get some basic info about each team
g.get_home_team()

CPU times: user 3.72 ms, sys: 551 µs, total: 4.27 ms
Wall time: 3.83 ms


,abbrev,city,division_id,final_standing,id,season_id,team_id,ulti_analytics_ext_id,name,ext_team_id,ls_team_id
0,CAR,Carolina,4,None,242,13,18,None,Flyers,flyers,f364b191-1101-4c86-959e-7ac9b2a0909b


In [17]:
%%time
# We can also get the rosters and which players were active
g.get_home_roster().head()

CPU times: user 6.97 ms, sys: 1.28 ms, total: 8.24 ms
Wall time: 7.33 ms


,id,team_season_id,player_id,jersey_number,first_name,last_name,ext_player_id,ls_player_id,active
0,10139,242,2889,0,Wilson,Matthews,wmatthews1,8bdcaa7f-035c-4f60-a1b3-2b5352519ada,False
1,10140,242,994,1,Justin,Allen,jallen,62b8c47d-76a7-4eae-a397-a313327407be,False
2,10141,242,1643,2,Sol,Yanuck,syanuck,4338c80e-2abd-40e8-a32a-0b95e347aba3,True
3,10142,242,2890,3,Michael,Lee,mlee1,d56afe48-2b65-4de1-8594-5650ea403a92,False
4,10143,242,2222,4,Seth,Weaver,sweaver,3264b70f-0b34-4733-a3e7-56173decc733,True


In [18]:
%%time
# Get player stats for this game
g.get_player_stats_by_game(home=True).head()

/Users/johnlithio/Documents/AUDL-Advanced-Stats/audl_advanced_stats/game.py:2380: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.

/Users/johnlithio/Documents/AUDL-Advanced-Stats/audl_advanced_stats/game.py:2781: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.



CPU times: user 1.14 s, sys: 11.9 ms, total: 1.15 s
Wall time: 1.14 s


,playerid,name,team,opponent,game_date,year,playoffs,games,total_points,o_points,...,receptions_dump_pct,completion_huck_pct,attempts_huck_pct,receptions_huck_pct,completion_swing_pct,attempts_swing_pct,receptions_swing_pct,completion_throw_pct,attempts_throw_pct,receptions_throw_pct
0,10141,Sol Yanuck,CAR,ATL,2022-04-29,2022,False,1,16,16,...,0.162791,0.5,0.081633,0.000000,1.0,0.204082,0.232558,1.0,0.408163,0.186047
1,10143,Seth Weaver,CAR,ATL,2022-04-29,2022,False,1,10,0,...,0.333333,NaN,0.000000,0.000000,1.0,0.200000,0.000000,1.0,0.400000,0.166667
2,10147,Terrence Mitchell,CAR,ATL,2022-04-29,2022,False,1,14,14,...,0.000000,NaN,0.000000,0.000000,NaN,0.000000,0.000000,1.0,0.250000,0.400000
3,10149,Henry Fisher,CAR,ATL,2022-04-29,2022,False,1,11,10,...,0.000000,1.0,0.083333,0.076923,1.0,0.083333,0.076923,1.0,0.083333,0.384615
4,10151,William Coffin,CAR,ATL,2022-04-29,2022,False,1,5,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
%%time
# Visualize the progression of scoring
g.visual_game_score()

CPU times: user 86.1 ms, sys: 8.23 ms, total: 94.3 ms
Wall time: 97.9 ms


In [20]:
%%time
# Visualize the substitution patterns
g.visual_game_flow()

CPU times: user 309 ms, sys: 7.29 ms, total: 316 ms
Wall time: 329 ms


In [22]:
%%time
# Visualize a single possession
g.visual_possession_map_vertical(possession_number=1)

CPU times: user 130 ms, sys: 24.5 ms, total: 154 ms
Wall time: 174 ms


In [4]:
import pandas as pd
home = True
if home:
    roster_str = "Home"
else:
    roster_str = "Away"

# Get list of dicts of roster
roster_raw = g.get_response()[f"rosters{roster_str}"]

# Convert to dataframe and un-nest dicts
roster = (
    pd.DataFrame.from_records(roster_raw)
    .merge(
        pd.DataFrame.from_records([x["player"] for x in roster_raw]),
        how="left",
        left_on=["player_id"],
        right_on=["id"],
        suffixes=["", "_y"],
    )
    .drop(columns=["id_y", "player", "active"])
    .drop_duplicates()
    .rename(columns={"id": "player_season_id", "player_id": "id"})
)

# Get active players
active_raw = g.response[f"tsg{roster_str}"]["rosterIds"]
active = pd.DataFrame(data=[[x] for x in active_raw], columns=["id"])
active["active"] = True

# Add column for whether players were active for this game
roster = roster.merge(active, how="left", on=["id"]).assign(
    active=lambda x: x["active"].fillna(False)
)
roster

,player_season_id,team_season_id,id,jersey_number,first_name,last_name,ext_player_id,ls_player_id,active
0,9452,210,2001,8,TJ,Stanton,tstanton,4a113253-fcab-462d-b36d-0cd15f955a7f,False
1,9451,210,174,4,Ben,Jagt,bjagt,6d4fbe3d-da8a-4390-b082-5b299b367c49,False
2,9450,210,2093,3,Elliott,Chartock,echartock,cb680971-9250-4928-8f16-297f7ea06352,False
3,9449,210,1545,1,Ryan,Saffa,rsaffa,559bfb06-bffe-4a47-bee7-446dd89b4081,False
4,9448,210,1543,2,Ryan,Osgar,rosgar,eee9ed39-be6b-4697-808e-7383082a46d3,False
5,9453,210,2637,9,Solomon,Rueschemeyer-Bailey,srueschem,be6f0e2b-e914-4af4-857f-f4b707d3e9bd,False
6,9460,210,2638,22,Angelo,Olcese,aolcese,5e163bd4-0a35-4d69-ae16-9d08f55053b9,False
7,9466,210,2639,54,John,Lithio,jlithio,6b1d93f6-d9a7-4f88-a672-647f0d152c40,False
8,9457,210,874,14,Joe,Ouellette,jouellett,e08cd129-3c1c-45e1-9383-38a8e0bc1316,False
9,9456,210,1207,12,Matt,LeMar,mlemar,706838fb-5980-4b4b-a1e1-055f6c706b5e,False


In [7]:
player_names = (
    roster.assign(
        name=lambda x: x["first_name"].str.strip()
        + " "
        + x["last_name"].str.strip()
    )[["player_season_id", "id", "name"]]
    .set_index("id")
    .to_dict()["name"]
)
player_names

{2001: 'TJ Stanton',
 174: 'Ben Jagt',
 2093: 'Elliott Chartock',
 1545: 'Ryan Saffa',
 1543: 'Ryan Osgar',
 2637: 'Solomon Rueschemeyer-Bailey',
 2638: 'Angelo Olcese',
 2639: 'John Lithio',
 874: 'Joe Ouellette',
 1207: 'Matt LeMar',
 747: 'Jack Williams',
 408: 'Conor Kline',
 1314: 'Mike Drost',
 1221: 'Matt Stevens',
 1536: 'Ryan Holmes',
 2000: 'Noriaki To',
 178: 'Ben Katz',
 2293: 'Sam Feder',
 2075: 'Ryan Weaver',
 1227: 'Matt Weintraub',
 182: 'Ben Nelson',
 1531: 'Ryan Drost',
 1180: 'Marques Brownlee',
 823: 'Jeff Babbitt',
 963: 'Josue Alorro',
 2715: 'Kainoa Chun Moy',
 2716: 'Oliver Feder',
 2295: 'Kyle Dew',
 2717: 'Brooks Wallace',
 2710: 'Matt LaBar',
 2718: 'Elias Griffin',
 1343: "Nasser M'Bae Vogel"}

In [8]:
g.get_home_roster()

,player_season_id,team_season_id,id,jersey_number,first_name,last_name,ext_player_id,ls_player_id,active
0,9452,210,2001,8,TJ,Stanton,tstanton,4a113253-fcab-462d-b36d-0cd15f955a7f,False
1,9451,210,174,4,Ben,Jagt,bjagt,6d4fbe3d-da8a-4390-b082-5b299b367c49,False
2,9450,210,2093,3,Elliott,Chartock,echartock,cb680971-9250-4928-8f16-297f7ea06352,False
3,9449,210,1545,1,Ryan,Saffa,rsaffa,559bfb06-bffe-4a47-bee7-446dd89b4081,False
4,9448,210,1543,2,Ryan,Osgar,rosgar,eee9ed39-be6b-4697-808e-7383082a46d3,False
5,9453,210,2637,9,Solomon,Rueschemeyer-Bailey,srueschem,be6f0e2b-e914-4af4-857f-f4b707d3e9bd,False
6,9460,210,2638,22,Angelo,Olcese,aolcese,5e163bd4-0a35-4d69-ae16-9d08f55053b9,False
7,9466,210,2639,54,John,Lithio,jlithio,6b1d93f6-d9a7-4f88-a672-647f0d152c40,False
8,9457,210,874,14,Joe,Ouellette,jouellett,e08cd129-3c1c-45e1-9383-38a8e0bc1316,False
9,9456,210,1207,12,Matt,LeMar,mlemar,706838fb-5980-4b4b-a1e1-055f6c706b5e,False
